In [1]:
from mdagent import MDAgent


In [2]:
descriptive_prompt_3 = "I need you to help me with MD tasks:  1.- Download the PDB file 1ZNI 2.- Pre-process the file by Adding missing atoms, and replacing non-standard residues. 3.- Simulate for 1ps at 300 K"
non_descriptive_prompt_3 = "Simulate 1ZNI for 1ps at temp=300K"

In [3]:
llm_model = "gpt-4-turbo-2024-04-09"
tools = "all"

In [4]:
agent_1 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)
agent_2 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)

In [5]:
agent_1.run(descriptive_prompt_3)

Thought: The task involves multiple steps: downloading a PDB file, preprocessing it, and running a molecular dynamics simulation. I will start by downloading the PDB file for 1ZNI.

Action:
```json
{
    "action": "PDBFileDownloader",
    "action_input": {
        "query": "1ZNI"
    }
}
```PDB file found with this ID: 1ZNI
Now that the PDB file 1ZNI has been successfully downloaded, the next step is to preprocess this file. This involves adding missing atoms and replacing non-standard residues. I will use the CleaningToolFunction to perform these tasks.

Action:
```json
{
    "action": "CleaningToolFunction",
    "action_input": {
        "pdb_id": "1ZNI_130703",
        "add_missing_atoms": true,
        "replace_nonstandard_residues": true,
        "remove_heterogens": false,
        "remove_water": false,
        "add_hydrogens": false
    }
}
```file path: /Users/jorgemedina/Desktop/md-agent/ckpt/ckpt_356/pdb/1ZNI_raw_20240918_130703.pdb
The PDB file 1ZNI has been successfully dow

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [6]:
agent_2.run(non_descriptive_prompt_3)

Thought: To simulate the protein with PDB ID 1ZNI for 1 picosecond at a temperature of 300K, I need to set up and run a molecular dynamics simulation. I will use the SetUpandRunFunction tool to perform this task, specifying the necessary parameters for the simulation.

Action:
```json
{
    "action": "SetUpandRunFunction",
    "action_input": {
        "pdb_id": "1ZNI",
        "forcefield_files": ["amber99sb.xml", "tip3p.xml"],
        "system_params": {
            "nonbondedMethod": "PME",
            "nonbondedCutoff": "1.0 * nanometers",
            "constraints": "HBonds",
            "rigidWater": true,
            "solvate": true
        },
        "integrator_params": {
            "integrator_type": "LangevinMiddle",
            "Temperature": "300 * kelvin",
            "Friction": "1.0 / picoseconds",
            "Timestep": "0.002 * picoseconds"
        },
        "simulation_params": {
            "Ensemble": "NVT",
            "Number of Steps": 500,
            "record_

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [7]:
from mdagent import MDAgent
agent = MDAgent(agent_type="Structured", ckpt_dir="ckpt_180")
registry = agent.path_registry
print(registry.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1ZNI_192718: PDB file downloaded from RSCB
 PDBFile ID: 1ZNI_192718
 1ZNI_192720: Cleaned File:  Replaced Nonstandard Residues. Missing Atoms Added and replaces nonstandard residues. 
 1ZNI_192727: Cleaned File:  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 1ZNI_192733: Cleaned File:  Removed Heterogens
 and Water Kept.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_192738: Initial positions for simulation sim0_192738
 sim0_192738: Basic Simulation of Protein 1ZNI_192733
 rec0_192738: Simulation trajectory for protein 1ZNI_192733 and simulation sim0_192738
 rec1_192738: Simulation state log for protein 1ZNI_192733 and simulation sim0_192738
 rec2_192738: Simulation pdb frames for protein 1ZNI_192733 and simulation sim0_192738
 1ZNI_192801: PDB file downloaded from RSCB
 PDBFile ID: 1ZNI_192801
 1ZNI_192808: Cleaned 

In [8]:
import os 
import mdtraj as md
traj_path_1 = registry.get_mapped_path("rec0_192738")
top_path_1 = registry.get_mapped_path("top_sim0_192738")

traj_path_2 = registry.get_mapped_path("rec0_192811")
top_path_2 = registry.get_mapped_path("top_sim0_192811")

assert os.path.exists(traj_path_1)
assert os.path.exists(top_path_1)
assert os.path.exists(traj_path_2)
assert os.path.exists(top_path_2)

In [9]:
traj_1 = md.load(traj_path_1, top=top_path_1)
traj_2 = md.load(traj_path_2, top=top_path_2)
print(traj_1.n_frames)
print(traj_2.n_frames)

300
100


In [ ]:
print(traj_1.n_atoms)
print(traj_2.n_atoms)

In [ ]:
#residues before
raw_path = registry.get_mapped_path("1ZNI_192718")
clean_path = registry.get_mapped_path("1ZNI_192808")
print(raw_path, clean_path)
raw_file = md.load(raw_path)
clean_file = md.load(clean_path)
residues_before = set([str(res)[:3] for res in raw_file.top.residues])
residues_after = set([str(res)[:3] for res in clean_file.top.residues])
print("Residues before:",residues_before)
print("Residues after:",residues_after)
print("Residues removed:",residues_before - residues_after)